In [2]:
import pandas as pd
from torch.utils.data import DataLoader
from data_files.dataset import CQT_Dataset_test
import torch
from utils.model import Spice_model
import numpy as np

In [3]:
mir_test_data = pd.read_pickle("./data_test/MIR1k.pkl") 

In [4]:
mir_test_data

,0,1,2,3,4,5,6,7,8,9,...,183,184,185,186,187,188,189,190,191,192
0,0.000,0.004339,0.006039,0.009540,0.012700,0.012407,0.010001,0.008643,0.008276,0.007767,...,0.000153,0.000165,0.000164,0.000155,0.000151,0.000159,0.000168,0.000164,0.0,0.000000
1,0.032,0.004647,0.006073,0.009980,0.014301,0.013825,0.010343,0.009158,0.008959,0.008272,...,0.000036,0.000024,0.000039,0.000061,0.000061,0.000081,0.000088,0.000068,0.0,0.000000
2,0.064,0.004902,0.006037,0.010141,0.015858,0.015103,0.010261,0.009526,0.009474,0.008453,...,0.000062,0.000068,0.000063,0.000029,0.000041,0.000046,0.000042,0.000028,0.0,0.000000
3,0.096,0.005066,0.005998,0.009985,0.017362,0.016220,0.009778,0.009759,0.009816,0.008290,...,0.000169,0.000419,0.000270,0.000126,0.000104,0.000052,0.000077,0.000149,1.0,119.626230
4,0.128,0.005128,0.006015,0.009505,0.018816,0.017146,0.008939,0.009824,0.009996,0.007773,...,0.000283,0.000087,0.000316,0.000283,0.000053,0.000029,0.000184,0.000038,1.0,156.307519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251087,7.232,0.016996,0.024627,0.025763,0.022843,0.016272,0.014689,0.033259,0.042570,0.033413,...,0.001452,0.000392,0.000755,0.002354,0.001939,0.000348,0.000117,0.000349,1.0,0.000000
251088,7.264,0.015188,0.022720,0.023012,0.019354,0.013047,0.010927,0.029718,0.038772,0.030509,...,0.000944,0.000356,0.000100,0.000186,0.000224,0.000131,0.000146,0.000273,0.0,0.000000
251089,7.296,0.013305,0.020496,0.019982,0.015793,0.010059,0.007795,0.025998,0.034271,0.027372,...,0.000245,0.000061,0.000049,0.000027,0.000074,0.000071,0.000051,0.000050,0.0,0.000000
251090,7.328,0.011442,0.018007,0.016781,0.012372,0.007500,0.005803,0.022253,0.029385,0.024091,...,0.000109,0.000050,0.000045,0.000013,0.000049,0.000024,0.000034,0.000038,0.0,0.000000


In [5]:
batch_size = 64
mir_test_batches = DataLoader(CQT_Dataset_test(data=mir_test_data, mode='test'), batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [14]:
for inputs, targets in enumerate(mir_test_batches):
    int_shift = targets[1]
    inp1 = targets[2]
    inp2 = targets[3]
    label = np.vstack((targets[4].detach().numpy(),targets[5].detach().numpy()))
    if inputs == 4:
        break

In [6]:
# load model trained on MDB
model = Spice_model([1, 64, 128, 256, 512, 512, 512], [512, 512, 512, 256, 128, 64, 1], [True, True, True, True, True, True])
checkpoint = torch.load('./rev_1k_checkpoints/checkpoint_mir_643.ckp', 'cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [7]:
def output2hz(pitch_output):
  # Constants taken from https://tfhub.dev/google/spice/2
  PT_OFFSET = 25.58
  PT_SLOPE = 63.07
  FMIN = 10.0    #why is it 10, not 110?
  BINS_PER_OCTAVE = 12.0  
  cqt_bin = pitch_output * PT_SLOPE + PT_OFFSET;
  return FMIN * 2.0 ** (1.0 * cqt_bin / BINS_PER_OCTAVE)

In [8]:
1 / (24 * np.log2(666.664939769901 /  66.9456331525636256))

0.012565718708847283

In [9]:
import mir_eval
from tqdm import tqdm

def rpa_on_dataset(model, batches, sigma=0.0125657):
    pred_pitch = np.array([])
    pred_pitch_cent = np.array([])
    pred_pitch_voicing = np.array([])
    lab_pitch_cent = np.array([])
    lab_pitch_voicing = np.array([])
    for inputs, targets in enumerate(tqdm(batches)):
        int_shift = targets[1].detach().numpy()
        inp1 = targets[2]
        inp2 = targets[3]
        label_voice = targets[4].detach().numpy()
        label_f0 = targets[5].detach().numpy()
        pitch_h1,conf_h1,x_hat1 = model(inp1.float())
        pitch_h2,conf_h2,x_hat2 = model(inp2.float())
        abs_pitch1 = np.apply_along_axis(output2hz, 0, pitch_h1.detach().numpy())
        abs_pitch2 = np.apply_along_axis(output2hz, 0, pitch_h2.detach().numpy())
        # abs_pitch_cent1 = np.apply_along_axis(mir_eval.melody.hz2cents, 0, abs_pitch1))
        # abs_pitch_cent2 = np.apply_along_axis(mir_eval.melody.hz2cents, 0, abs_pitch2))
        pitch_diff =  np.abs((abs_pitch1-abs_pitch2) - (sigma*int_shift).reshape(int_shift.shape[0],1))
        # pitch_diff_cent =  np.abs(abs_pitch_cent1-abs_pitch_cent2)
        # pitch_diff[pitch_diff>0.5] #not even a sigle one even without semitone implementation
        
        ## implemented on average pitch
        pred_pitch_batch = np.mean([abs_pitch1, abs_pitch2], axis=0)
        pred_pitch = np.append(pred_pitch, pred_pitch_batch)
        temp = np.apply_along_axis(mir_eval.melody.hz2cents, 0, pred_pitch_batch)
        pred_pitch_cent = np.append(pred_pitch_cent, temp)
        pred_pitch_voicing = np.append(pred_pitch_voicing,label_voice) #replace with conf head
        lab_pitch_cent = np.append(lab_pitch_cent,np.apply_along_axis(mir_eval.melody.hz2cents, 0, label_f0))
        lab_pitch_voicing = np.append(lab_pitch_voicing,label_voice) 
        if inputs == 5:
            break
        print("RPA : {} after batch {}".format(mir_eval.melody.raw_pitch_accuracy(lab_pitch_voicing, lab_pitch_cent, pred_pitch_voicing, pred_pitch_cent), inputs))
    raw_pitch_accuracy = mir_eval.melody.raw_pitch_accuracy(lab_pitch_voicing, lab_pitch_cent, pred_pitch_voicing, pred_pitch_cent)
    return raw_pitch_accuracy

In [10]:
rpa_on_dataset(model, mir_test_batches)

  0%|          | 1/3924 [00:01<1:39:56,  1.53s/it]

RPA : 0.0 after batch 0


  0%|          | 2/3924 [00:01<57:21,  1.14it/s]  

RPA : 0.0 after batch 1


  0%|          | 3/3924 [00:02<39:53,  1.64it/s]

...

RPA : 0.0 after batch 3


  0%|          | 5/3924 [00:02<27:09,  2.41it/s]

RPA : 0.0 after batch 4


  0%|          | 5/3924 [00:03<41:17,  1.58it/s]


0.0

In [136]:
y_hat = np.apply_along_axis(output2hz, 0, pitch_h.detach().numpy())
y_hat_cent = np.apply_along_axis(mir_eval.melody.hz2cents, 0, y_hat)
y_hat_voice = np.random.randint(2, size=y_hat.shape)
y = np.random.uniform(low=40, high=65, size=(10,1)) #extrapolate back to time
y_cent = np.apply_along_axis(mir_eval.melody.hz2cents, 0, y)
y_voice = np.random.randint(2, size=y.shape)
raw_pitch_accuracy = mir_eval.melody.raw_pitch_accuracy(y_voice, y_cent, y_hat_voice, y_hat_cent)
raw_pitch_accuracy

NameError: name 'pitch_h' is not defined

In [16]:
pitch_h1,conf_h1,x_hat1 = model(inp1.float())
pitch_h2,conf_h2,x_hat2 = model(inp2.float())

In [17]:
y_hat1 = np.apply_along_axis(output2hz, 0, pitch_h1.detach().numpy())
y_hat2 = np.apply_along_axis(output2hz, 0, pitch_h2.detach().numpy())

In [22]:
print("{}\n{}\n{}\n{}".format(y_hat1,y_hat2,label,int_shift))

[[39.118362]
 [34.591106]
 [42.45288 ]
 [38.605713]
 [41.83369 ]
 [39.445374]
 [43.073883]
 [41.75791 ]
 [40.20938 ]
 [39.736454]
 [36.421143]
 [39.377724]
 [36.874325]
 [42.704144]
 [43.76687 ]
 [39.899994]
 [37.150597]
 [39.383377]
 [38.50537 ]
 [43.74185 ]
 [38.253216]
 [38.49774 ]
 [40.302807]
 [39.321465]
 [41.837482]
 [41.68345 ]
 [43.408997]
 [39.283783]
 [38.699707]
 [43.730865]
 [38.888096]
 [41.08494 ]
 [39.74883 ]
 [43.606888]
 [39.970196]
 [43.769325]
 [39.447826]
 [39.422005]
 [42.45598 ]
 [38.739914]
 [39.44508 ]
 [43.73856 ]
 [43.310112]
 [38.2805  ]
 [41.843666]
 [36.92688 ]
 [43.77212 ]
 [42.913223]
 [39.805077]
 [40.221245]
 [39.030243]
 [43.75689 ]
 [38.04477 ]
 [36.197372]
 [43.418674]
 [43.767548]
 [39.446075]
 [38.94567 ]
 [43.655354]
 [40.501286]
 [37.658596]
 [42.480026]
 [43.75673 ]
 [40.558205]]
[[39.223717]
 [35.33662 ]
 [42.83276 ]
 [38.562073]
 [41.9638  ]
 [39.75415 ]
 [43.280647]
 [41.6503  ]
 [40.223278]
 [39.751564]
 [36.447716]
 [39.14907 ]
 [36.88248 